In [3]:
pip install praw pandas matplotlib wordcloud nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pytesseract pillow langdetect

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR"

In [12]:
!pip install transformers

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.0 MB 2.8 MB/s eta 0:00:04
   ------ --------------------------------- 1.8/12.0 MB 3.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.6/12.0 MB 3.5 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/12.0 MB 4.0 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.0 MB 4.3 MB/s eta 0:00:02
   -------------------- ------------------- 6.0/12.0 MB 4.6 MB/s eta 0:00:02
   ------------------------ --------------- 7.3/12.0 MB 4.6 MB/s eta 0:00:02
   ---------------------------- ----------- 8.7/12.0 MB 4.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 5.0 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 5.2 MB/s eta 0:00:00
   -------

In [14]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl (2817.2 MB)
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.5 MB 3.3 MB/s eta 0:00:02
   --------- ------------------------------ 1.3/5.5 MB 4.2 MB/s eta 0:00:01
   ------------------- -------------------- 2.6/5.5 MB 5.0 MB/s eta 0:00:01
   ---------------------------- ----------- 3.9/5.5 MB 5.4 MB/s eta 0:00:01
   ------------------------------------ --- 5.0/5.5 MB 5.4 MB/s eta 0:00:01
   --------------

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.7.1+cu118
True


# Commit to GitHub

In [ ]:
!pwd                # shows your current folder
!git status         # check uncommitted changes
!git add .
!git commit -m "Scraping post and comment data"
!git push origin main

# Import Statements

In [15]:
import os
import praw
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
import logging
import time
import matplotlib as plt
from wordcloud import WordCloud
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import pytesseract
import io
import torch


# Reddit API checking

In [8]:
import praw

reddit = praw.Reddit(
    client_id="3VypVlHsKWYOd43a4XZ1Qw",            # your client ID
    client_secret="RnwapkXWjkuee9Rv8B5rEoaOscL4wg", # your secret
    user_agent="NLP_SriLankaResearchApp by u/Cute_Firefighter1926"
)


In [9]:
print(reddit.user.me())   # should print "None" (since it's read-only)
for post in reddit.subreddit("srilanka").hot(limit=5):
    print(post.title)

None
Why aren’t true religious leaders questioning this?
In case if you wanted concrete proof who was behind yesterday's Trinco incident 😂
Can't believe "THE C*NT OF MONTE CRISTO" is in stock at Sarasavi Book Shop 😊
a $1m for a $250k car is crazy.
In a conflict between India and Pakistan who do you support?


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tharu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
subreddit_name = "srilanka"
sub = reddit.subreddit(subreddit_name)

# Pipeline

## Pipeline to collect data from Posts

### Important Functions

In [7]:
# Function to make sure the sentence is meaningful (words more than 4 words)

def valid_text(text):
    """Check if text exists and has >4 words."""
    return isinstance(text, str) and len(text.split()) > 4

In [8]:
# Function to collect all the posts from "new"

def collect_posts_from_new():
    """Collect all available latest posts from the subreddit"""
    data = []
    print("Collecting all available latest posts...")

    for post in sub.new(limit=None):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "new",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,  # ✅ Save subreddit name
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} posts.")
    return data

In [9]:
# Collect posts from "hot"

def collect_posts_from_hot(limit=None):
    """Collect all available hot posts from the subreddit"""
    data = []
    print("Collecting all hot posts...")

    for post in sub.hot(limit=limit):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "hot",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} hot posts.")
    return data

In [10]:
# Collect posts from "top"

def collect_posts_from_top(limit=None, time_filter="all"):
    """Collect top posts from the subreddit with optional time_filter"""
    data = []
    print(f"Collecting top posts (time_filter='{time_filter}')...")

    for post in sub.top(limit=limit, time_filter=time_filter):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "top",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} top posts.")
    return data


In [11]:
# Function to collect all the posts from search for each keyword

def collect_posts_from_search(keywords):
    """Collect posts via keyword search"""
    data = []
    for kw in keywords:
        print(f"🔎 Searching for keyword: {kw}")
        try:
            for post in sub.search(kw, sort="new", time_filter="all", limit=None):
                content = f"{post.title} {post.selftext}".strip()
                if valid_text(content):
                    data.append({
                        "type": "post",
                        "source": "search",
                        "keyword": kw,
                        "id": post.id,
                        "created_utc": post.created_utc,
                        "author": str(post.author),
                        "subreddit": post.subreddit.display_name,  # ✅ Save subreddit name
                        "content": content,
                        "score": post.score,
                        "num_comments": post.num_comments
                    })
            print(f"✅ Done with keyword '{kw}', total so far: {len(data)}")
        except Exception as e:
            print(f"⚠️ Error with keyword '{kw}': {e}")
        time.sleep(2)  # avoid rate limits
    return data

### Collecting Data

In [12]:
# Collect from .new() 

posts_new = collect_posts_from_new()

Collected 997 posts.


In [13]:
# Collect from .hot() 

posts_hot = collect_posts_from_hot()

Collected 998 hot posts.


In [14]:
# Collect from .top() 

posts_top = collect_posts_from_top()

Collected 231 top posts.


In [15]:
# Collect from .search() using "Keywords"

keywords = [
    # --- Politics and Governance ---
    "politics", "government", "parliament", "president", "prime minister", "minister", "election", 
    "voting", "policy", "corruption", "protest", "democracy", "dictatorship", "rights", 
    "foreign relations", "IMF", "debt", "reform", "budget cuts", "privatization", "diplomacy",
    "public administration", "bureaucracy", "governance", "development plan",

    # --- Economy and Employment ---
    "economy", "inflation", "finance", "budget", "tax", "trade", "business", "industry",
    "investment", "employment", "unemployment", "job", "market", "entrepreneurship", "startups", 
    "SMEs", "cost of living", "poverty", "income", "wages", "labor", "productivity",

    # --- Education and Learning ---
    "education", "school", "preschool", "primary education", "secondary education", "college", 
    "university", "campus", "faculty", "higher education", "studies", "student", "students", 
    "teacher", "lecturer", "professor", "principal", "tuition", "scholarship", "exam", "O/L", "A/L",
    "curriculum", "syllabus", "textbook", "subject", "education reform", "education policy", 
    "private school", "public school", "international school", "ministry of education", 
    "education inequality", "literacy", "dropout", "school fees", "admission", 
    "online learning", "distance learning", "e-learning", "virtual classroom", 
    "digital education", "EdTech", "learning app", "AI in education", "STEM", "ICT", 
    "vocational training", "technical education", "TVET", "teacher training", 
    "exam results", "Z-score", "university admission", "research", "academic", "publication", 
    "school reopening", "school closure", "classroom", "homework", "discipline", "student union", 
    "education budget", "education crisis", "education rights",

    # --- Health and Well-being ---
    "health", "hospital", "doctor", "nurse", "medicine", "pharmacy", "covid", 
    "mental health", "nutrition", "vaccination", "public health", "epidemic", "wellness",

    # --- Transport and Infrastructure ---
    "transport", "bus", "train", "traffic", "road", "airport", "port", "highway", 
    "vehicle", "fuel", "petrol", "transportation policy", "traffic jam",

    # --- Tourism and Culture ---
    "tourism", "travel", "visa", "hotel", "heritage", "beach", "wildlife", "culture", 
    "festival", "tradition", "food", "art", "music", "dance", "religion", "temple", "church", "mosque",

    # --- Society and Community ---
    "society", "community", "religion", "ethnicity", "language", "gender", "family", 
    "youth", "marriage", "divorce", "social media", "influence", "violence", "addiction",

    # --- Technology and Innovation ---
    "technology", "internet", "mobile", "AI", "machine learning", "robotics", 
    "startup", "innovation", "science", "engineering", "research", "data science", 
    "cybersecurity", "software", "coding", "programming", "app", "digital divide",

    # --- Environment and Agriculture ---
    "environment", "climate", "pollution", "forest", "wildlife", "sustainability", 
    "agriculture", "farming", "paddy", "organic farming", "flood", "drought", "waste", 
    "recycling", "deforestation",

    # --- Sports and Recreation ---
    "sports", "cricket", "football", "rugby", "athletics", "games", "Olympics", 
    "tournament", "fitness", "training", "coach", "sports development",

    # --- Development and Infrastructure ---
    "development", "infrastructure", "housing", "urban", "construction", "energy", 
    "electricity", "renewable energy", "water", "sanitation", "drainage", "smart city",

    # --- Law and Order ---
    "law", "justice", "court", "crime", "police", "rights", "freedom", 
    "protest", "migration", "refugee", "diaspora", "human rights",

    # --- Media and Communication ---
    "media", "journalism", "press", "television", "cinema", "radio", 
    "music", "art", "literature", "publication", "podcast",
    "communication", "telecom", "Dialog", "SLT", "internet speed", 
    "data", "mobile network", "social network", "broadcast",

    # --- Defense and Security ---
    "military", "security", "terrorism", "war", "LTTE", "peace", 
    "national security", "army", "navy", "air force",

    # --- Society ---
    "entertainment", "movies", "gaming", "memes", "news", "update", "local event", 
    "weather", "rain", "heatwave", "temperature", "food prices", 
    "charity", "donation", "public service", "volunteer", "NGO", "social work"
]

posts_search = collect_posts_from_search(keywords)

🔎 Searching for keyword: politics
✅ Done with keyword 'politics', total so far: 228
🔎 Searching for keyword: government
✅ Done with keyword 'government', total so far: 475
🔎 Searching for keyword: parliament
✅ Done with keyword 'parliament', total so far: 696
🔎 Searching for keyword: president
✅ Done with keyword 'president', total so far: 930
🔎 Searching for keyword: prime minister
✅ Done with keyword 'prime minister', total so far: 1039
🔎 Searching for keyword: minister
✅ Done with keyword 'minister', total so far: 1264
🔎 Searching for keyword: election
✅ Done with keyword 'election', total so far: 1492
🔎 Searching for keyword: voting
✅ Done with keyword 'voting', total so far: 1719
🔎 Searching for keyword: policy
✅ Done with keyword 'policy', total so far: 1954
🔎 Searching for keyword: corruption
✅ Done with keyword 'corruption', total so far: 2172
🔎 Searching for keyword: protest
✅ Done with keyword 'protest', total so far: 2387
🔎 Searching for keyword: democracy
✅ Done with keywor

In [18]:
# Combine post datasets 

all_posts = posts_new + posts_hot + posts_top + posts_search
df_posts = pd.DataFrame(all_posts)

In [19]:
print("Total posts collected (without handling duplicates): ", len(df_posts))
df_posts.head(10)

Total posts collected (without handling duplicates):  43655


,type,source,keyword,id,created_utc,author,subreddit,content,score,num_comments
0,post,new,None,1otaemb,1.762771e+09,Cookiehere6969,srilanka,Is this a Scam or good investment? Haritha Lan...,2,1
1,post,new,None,1otaam5,1.762770e+09,oshan789,srilanka,Villa units for sale in Unawatuna Sri Lanka ! ...,3,0
2,post,new,None,1ot9w1v,1.762769e+09,mgssjjsks,srilanka,Whats your hot take on Sri Lanka as the title ...,3,8
3,post,new,None,1ot9kwe,1.762768e+09,Critical_Rise_exe,srilanka,Is the rs.11 deals real in Daraz?,1,3
4,post,new,None,1ot9h2f,1.762767e+09,No-Leave8971,srilanka,Need advice from the experts 🙏 [](https://www....,2,0
5,post,new,None,1ot9dyw,1.762767e+09,hotstar10,srilanka,Confusion Over Paddock Club Nugegoda’s Halal S...,0,4
6,post,new,None,1ot9da2,1.762767e+09,prav_u,srilanka,Tour to Kanneliya Rain Forest I’m planning a g...,1,1
7,post,new,None,1ot8r8a,1.762764e+09,Unusual-Witness-7304,srilanka,"ChatGPT vs electricians, my house wiring is no...",6,5
8,post,new,None,1ot8lir,1.762764e+09,negative-impactr8888,srilanka,SLTMobitel changed the superuser router passwo...,3,0
9,post,new,None,1ot8l95,1.762764e+09,Jakesbond007,srilanka,Can we hand carry medicine from abroad into Sr...,1,3


In [20]:
df_posts.drop_duplicates(subset=["content"], inplace=True)
print(f"Total posts collected (unique): {len(df_posts)}")

Total posts collected (unique): 18840


In [21]:
# Save df_posts to a CSV file
df_posts.to_csv("df_posts.csv", index=False)  

## Pipeline to collect data from Comments

In [22]:
# Function to collect first 20 comments for up to 6000 posts

def collect_comments_from_posts(post_ids, max_posts=10000, max_comments_per_post=50):
    """Fetch the first N comments for up to max_posts post IDs"""
    comments_data = []
    counter = 0

    # Limit the number of posts to max_posts
    post_ids = post_ids[:max_posts]
    print(f"Collecting comments for {len(post_ids)} posts (up to {max_comments_per_post} comments each)...")

    for pid in post_ids:
        counter += 1

        try:
            submission = reddit.submission(id=pid)
            # Fetch only the top-level comments (no "load more")
            submission.comments.replace_more(limit=0)  
            comments_list = submission.comments.list()[:max_comments_per_post]  # first 30 comments

            for comment in comments_list:
                if valid_text(comment.body):
                    comments_data.append({
                        "type": "comment",
                        "source": "comment",
                        "keyword": None,
                        "id": comment.id,
                        "parent_post": pid,
                        "created_utc": comment.created_utc,
                        "author": str(comment.author),
                        "subreddit": comment.subreddit.display_name,  
                        "content": comment.body,
                        "score": comment.score,
                        "num_comments": None
                    })

            if counter % 100 == 0:
                print(f"Processed {counter} posts... total comments so far: {len(comments_data)}")

            time.sleep(2)  # avoid rate limiting

        except Exception as e:
            print(f"⚠️ Error fetching comments for {pid}: {e}")
            time.sleep(5)

    df_comments = pd.DataFrame(comments_data)
    print(f"💬 Total comments collected: {len(df_comments)}")
    return comments_data

In [23]:
# Collect comments 

comments = collect_comments_from_posts(df_posts["id"])

✅ Processed 100 posts... total comments so far: 593
✅ Processed 200 posts... total comments so far: 1311
✅ Processed 300 posts... total comments so far: 2048
✅ Processed 400 posts... total comments so far: 2759
✅ Processed 500 posts... total comments so far: 3412
✅ Processed 600 posts... total comments so far: 4105
✅ Processed 700 posts... total comments so far: 4824
✅ Processed 800 posts... total comments so far: 5705
✅ Processed 900 posts... total comments so far: 6370
✅ Processed 1000 posts... total comments so far: 7119
✅ Processed 1100 posts... total comments so far: 8634
✅ Processed 1200 posts... total comments so far: 10152
✅ Processed 1300 posts... total comments so far: 11403
✅ Processed 1400 posts... total comments so far: 12726
✅ Processed 1500 posts... total comments so far: 13786
✅ Processed 1600 posts... total comments so far: 14841
✅ Processed 1700 posts... total comments so far: 15957
✅ Processed 1800 posts... total comments so far: 16959
✅ Processed 1900 posts... total

In [24]:
df_comments = pd.DataFrame(comments)
len(df_comments)

54042

## Combine Posts and Comments

In [25]:
df_text = pd.concat([df_posts, df_comments], ignore_index=True)
df_text.drop_duplicates(subset=["content"], inplace=True)

In [28]:
df_text.head(10)

,type,source,keyword,id,created_utc,author,subreddit,content,score,num_comments,parent_post
0,post,new,None,1otaemb,1.762771e+09,Cookiehere6969,srilanka,Is this a Scam or good investment? Haritha Lan...,2,1,NaN
1,post,new,None,1otaam5,1.762770e+09,oshan789,srilanka,Villa units for sale in Unawatuna Sri Lanka ! ...,3,0,NaN
2,post,new,None,1ot9w1v,1.762769e+09,mgssjjsks,srilanka,Whats your hot take on Sri Lanka as the title ...,3,8,NaN
3,post,new,None,1ot9kwe,1.762768e+09,Critical_Rise_exe,srilanka,Is the rs.11 deals real in Daraz?,1,3,NaN
4,post,new,None,1ot9h2f,1.762767e+09,No-Leave8971,srilanka,Need advice from the experts 🙏 [](https://www....,2,0,NaN
5,post,new,None,1ot9dyw,1.762767e+09,hotstar10,srilanka,Confusion Over Paddock Club Nugegoda’s Halal S...,0,4,NaN
6,post,new,None,1ot9da2,1.762767e+09,prav_u,srilanka,Tour to Kanneliya Rain Forest I’m planning a g...,1,1,NaN
7,post,new,None,1ot8r8a,1.762764e+09,Unusual-Witness-7304,srilanka,"ChatGPT vs electricians, my house wiring is no...",6,5,NaN
8,post,new,None,1ot8lir,1.762764e+09,negative-impactr8888,srilanka,SLTMobitel changed the superuser router passwo...,3,0,NaN
9,post,new,None,1ot8l95,1.762764e+09,Jakesbond007,srilanka,Can we hand carry medicine from abroad into Sr...,1,3,NaN


In [29]:
len(df_text)

70853

In [34]:
df_text.to_csv("r_srilanka_text_dataset.csv", index=False)
print(f"Final text dataset saved — {len(df_text)} rows total!")

Final text dataset saved — 70853 rows total!


## Pipeline to extract from non-textual data (image posts)

In [5]:
posts_df = pd.read_csv("r_srilanka_text_dataset.csv") 
posts_df.head()

C:\Users\tharu\AppData\Local\Temp\ipykernel_31260\237063977.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  posts_df = pd.read_csv("r_srilanka_text_dataset.csv")


,type,source,keyword,id,created_utc,author,subreddit,content,score,num_comments,parent_post
0,post,new,NaN,1otaemb,1.762771e+09,Cookiehere6969,srilanka,Is this a Scam or good investment? Haritha Lan...,2,1.0,NaN
1,post,new,NaN,1otaam5,1.762770e+09,oshan789,srilanka,Villa units for sale in Unawatuna Sri Lanka ! ...,3,0.0,NaN
2,post,new,NaN,1ot9w1v,1.762769e+09,mgssjjsks,srilanka,Whats your hot take on Sri Lanka as the title ...,3,8.0,NaN
3,post,new,NaN,1ot9kwe,1.762768e+09,Critical_Rise_exe,srilanka,Is the rs.11 deals real in Daraz?,1,3.0,NaN
4,post,new,NaN,1ot9h2f,1.762767e+09,No-Leave8971,srilanka,Need advice from the experts 🙏 [](https://www....,2,0.0,NaN


In [12]:
def collect_image_post_urls(max_posts=500, batch_size=50):
    urls = []
    collected = 0

    while collected < max_posts:
        try:
            for post in sub.new(limit=batch_size):
                if collected >= max_posts:
                    break
                url = post.url
                # Check if URL is an image
                if any(url.lower().endswith(ext) for ext in [".jpg", ".jpeg", ".png"]) \
                    or "i.redd.it" in url.lower() or "imgur.com" in url.lower():
                    urls.append({
                        "id": post.id,
                        "url": url,
                        "title": post.title,
                        "author": str(post.author),
                        "created_utc": post.created_utc
                    })
                    collected += 1
            print(f"Collected {collected}/{max_posts} URLs...")
            time.sleep(2)  # sleep to avoid rate limits
        except Exception as e:
            print(f"⚠️ Connection error: {e}. Retrying in 5s...")
            time.sleep(5)
    return pd.DataFrame(urls)

# Collect URLs
image_urls_df = collect_image_post_urls(max_posts=500)
image_urls_df.to_csv("image_urls.csv", index=False)

Collected 8/500 URLs...
Collected 16/500 URLs...
Collected 24/500 URLs...
Collected 32/500 URLs...
Collected 40/500 URLs...
Collected 48/500 URLs...
Collected 56/500 URLs...
Collected 64/500 URLs...
Collected 72/500 URLs...
Collected 80/500 URLs...
Collected 88/500 URLs...
Collected 96/500 URLs...
Collected 104/500 URLs...
Collected 112/500 URLs...
Collected 120/500 URLs...
Collected 128/500 URLs...
Collected 136/500 URLs...
Collected 144/500 URLs...
Collected 152/500 URLs...
Collected 160/500 URLs...
Collected 168/500 URLs...
Collected 176/500 URLs...
Collected 184/500 URLs...
Collected 192/500 URLs...
Collected 200/500 URLs...
Collected 208/500 URLs...
Collected 216/500 URLs...
Collected 224/500 URLs...
Collected 232/500 URLs...
Collected 240/500 URLs...
Collected 248/500 URLs...
Collected 256/500 URLs...
Collected 264/500 URLs...
Collected 272/500 URLs...
Collected 280/500 URLs...
Collected 288/500 URLs...
Collected 296/500 URLs...
Collected 304/500 URLs...
Collected 312/500 URLs...

In [13]:
image_urls_df.head()

,id,url,title,author,created_utc
0,1ozh1f2,https://i.redd.it/2sety1w1qt1g1.jpeg,We are so done for bruhhh😭😭,shxf_1,1.763388e+09
1,1ozfhgk,https://i.redd.it/lsctz8c0et1g1.png,In case if you wanted concrete proof who was b...,curiouscolombite,1.763384e+09
2,1ozebo7,https://i.redd.it/rz788vim3t1g1.jpeg,In a conflict between India and Pakistan who d...,Dhanagg,1.763381e+09
3,1ozdn89,https://i.redd.it/o55zys1mws1g1.jpeg,Ideas on this Stand up/Comedy show?,Unreal_realist-7381,1.763378e+09
4,1ozdi55,https://i.redd.it/g2zyk41tus1g1.png,"Can't believe ""THE C*NT OF MONTE CRISTO"" is in...",wiknew1,1.763378e+09


In [17]:
# Initialize BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [18]:
# BLIP Captioning
def get_image_caption(url):
    try:
        img = Image.open(requests.get(url, stream=True, timeout=10).raw).convert("RGB")
        inputs = processor(img, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs, max_length=40)
        caption = processor.decode(output[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Error generating caption for {url}: {e}")
        return None

In [19]:
# OCR text extraction
def get_image_text(url):
    try:
        img = Image.open(requests.get(url, stream=True, timeout=10).raw).convert("RGB")
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        print(f"Error extracting text for {url}: {e}")
        return None

In [ ]:
# Filter posts with valid image URLs
mask = image_urls_df['url'].notna() & image_urls_df['url'].str.endswith(('.jpg', '.jpeg', '.png'))

# Generate captions
image_urls_df.loc[mask, 'image_caption'] = image_urls_df.loc[mask, 'url'].apply(get_image_caption)

# Extract text from images
image_urls_df.loc[mask, 'image_text'] = image_urls_df.loc[mask, 'url'].apply(get_image_text)

Error extracting text for https://i.redd.it/2sety1w1qt1g1.jpeg: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/lsctz8c0et1g1.png: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/rz788vim3t1g1.jpeg: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/o55zys1mws1g1.jpeg: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/g2zyk41tus1g1.png: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/giosjowiis1g1.jpeg: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text for https://i.redd.it/rgeqe9197s1g1.png: tesseract

In [ ]:
image_urls_df.head()

In [ ]:
# check the structure of the dataframe
# combine with whole dataset
#check for duplicates